In [9]:
#Simulation
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, mean_squared_error, mean_absolute_error
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from scipy.stats import multivariate_normal




#Scenario 1 - Independent Covariate Case

#Initialize
p = 5  
n_train = 1000  
n_test = 5000  
pi_plus = 0.05  #Minority class proportion
outer_iterations = 5
inner_iterations = 500

#Generation
def generate_data(n, mu_0, sigma_0, mu_1, sigma_1, pi_plus):
    n_minority = int(n * pi_plus)
    n_majority = n - n_minority
    X_majority = np.random.normal(mu_0, sigma_0, size=(n_majority, p))
    X_minority = np.random.normal(mu_1, sigma_1, size=(n_minority, p))
    X = np.vstack((X_majority, X_minority))
    y = np.hstack((np.zeros(n_majority), np.ones(n_minority)))
    return X, y


#Compute the theoretical posterior using the Bayes' Theorem
def compute_posterior(X, mu_0, Sigma_0, mu_1, Sigma_1, pi_plus):
    pi_minus = 1 - pi_plus
    p_x_given_y1 = multivariate_normal.pdf(X, mean=mu_1, cov=Sigma_1)
    p_x_given_y0 = multivariate_normal.pdf(X, mean=mu_0, cov=Sigma_0)
    p_y1_given_x = (p_x_given_y1 * pi_plus) / (p_x_given_y1 * pi_plus + p_x_given_y0 * pi_minus)
    return p_y1_given_x

all_results = []


#Outer loop across different DGP
for outer in range(outer_iterations):
    print(f"\n========== Outer Iteration {outer+1}/{outer_iterations} ==========\n")

    np.random.seed(outer)
    #mean
    mu_0 = np.random.uniform(1, 5, p)
    mu_1 = np.random.uniform(4, 6, p)

    #SE for each p
    sigma0_vec = np.random.uniform(2, 5, p)   
    sigma1_vec = np.random.uniform(1, 2, p)
    #Covariance
    Sigma_0 = np.diag(sigma0_vec ** 2)      
    Sigma_1 = np.diag(sigma1_vec ** 2)
   

    X_test, y_test = generate_data(n_test, mu_0, sigma0_vec, mu_1, sigma1_vec, pi_plus)
    theoretical_probabilities = compute_posterior(X_test, mu_0, Sigma_0, mu_1, Sigma_1, pi_plus)


    inner_results = []

    # Inner loop under fixed distribution
    for inner in tqdm(range(inner_iterations), desc=f"Outer Iteration {outer+1} Progress"):
        X_train, y_train = generate_data(n_train, mu_0, sigma0_vec, mu_1, sigma1_vec, pi_plus)


        
        # Train models using original, oversampled, undersampled, and weighted strategies
        model_orig = LogisticRegression(penalty=None, max_iter=1000).fit(X_train, y_train)
        X_over, y_over = RandomOverSampler(random_state=inner).fit_resample(X_train, y_train)
        model_over = LogisticRegression(penalty=None, max_iter=1000).fit(X_over, y_over)
        X_under, y_under = RandomUnderSampler(random_state=inner).fit_resample(X_train, y_train)
        model_under = LogisticRegression(penalty=None, max_iter=1000).fit(X_under, y_under)
        model_weighted = LogisticRegression(penalty=None, max_iter=1000, class_weight='balanced').fit(X_train, y_train)

      
        original_predicted_probabilities = model_orig.predict_proba(X_test)[:, 1]
        over_predicted_probabilities = model_over.predict_proba(X_test)[:, 1]
        under_predicted_probabilities = model_under.predict_proba(X_test)[:, 1]
        weighted_predicted_probabilities = model_weighted.predict_proba(X_test)[:, 1]

        # Probability Correction
        proportion_0 = (y_train == 0).sum() / len(y_train)
        proportion_1 = (y_train == 1).sum() / len(y_train)
        rate = proportion_0 / proportion_1
        adj_prob_over = over_predicted_probabilities / (rate - rate * over_predicted_probabilities + over_predicted_probabilities)
        rate2 = proportion_1 / proportion_0
        adj_prob_under = (rate2 * under_predicted_probabilities) / (rate2 * under_predicted_probabilities - under_predicted_probabilities + 1)
        adj_prob_weighted = (weighted_predicted_probabilities / proportion_0) / ((1 / proportion_1) + (1 / proportion_0) * weighted_predicted_probabilities - (1 / proportion_1) * weighted_predicted_probabilities)

        # Evaluation
        mse_results = {
            "Method": ["Original", "Oversampled", "Oversampled-Corr", "Undersampled", "Undersampled-Corr", "Weighted", "Weighted-Corr"],
            "MSE": [
                mean_squared_error(theoretical_probabilities, original_predicted_probabilities),
                mean_squared_error(theoretical_probabilities, over_predicted_probabilities),
                mean_squared_error(theoretical_probabilities, adj_prob_over),
                mean_squared_error(theoretical_probabilities, under_predicted_probabilities),
                mean_squared_error(theoretical_probabilities, adj_prob_under),
                mean_squared_error(theoretical_probabilities, weighted_predicted_probabilities),
                mean_squared_error(theoretical_probabilities, adj_prob_weighted),
            ],
            "MAE": [
                mean_absolute_error(theoretical_probabilities, original_predicted_probabilities),
                mean_absolute_error(theoretical_probabilities, over_predicted_probabilities),
                mean_absolute_error(theoretical_probabilities, adj_prob_over),
                mean_absolute_error(theoretical_probabilities, under_predicted_probabilities),
                mean_absolute_error(theoretical_probabilities, adj_prob_under),
                mean_absolute_error(theoretical_probabilities, weighted_predicted_probabilities),
                mean_absolute_error(theoretical_probabilities, adj_prob_weighted),
            ],
            "AUC": [
                roc_auc_score(y_test, original_predicted_probabilities),
                roc_auc_score(y_test, over_predicted_probabilities),
                roc_auc_score(y_test, adj_prob_over),
                roc_auc_score(y_test, under_predicted_probabilities),
                roc_auc_score(y_test, adj_prob_under),
                roc_auc_score(y_test, weighted_predicted_probabilities),
                roc_auc_score(y_test, adj_prob_weighted),
            ],
        }

        inner_results.append(pd.DataFrame(mse_results))

   
    df_results = pd.concat(inner_results)
    df_results["Outer"] = outer + 1
    summary = df_results.drop(columns=["Outer"]).groupby("Method").agg(["mean", "std"])

    print("\n=== Summary for Outer Iteration", outer+1, "===\n")
    print(summary)
    all_results.append(df_results)



df_all = pd.concat(all_results, ignore_index=True)[["Method", "MSE", "MAE", "AUC"]]

final_summary = df_all.groupby("Method").agg(["mean", "std"])
print("\n=== Final summary over all outers and inners ===\n")
print(final_summary)



========== Outer Iteration 1/5 ==========



Outer Iteration 1 Progress: 100%|██████████| 500/500 [00:16<00:00, 30.89it/s]



=== Summary for Outer Iteration 1 ===

                        MSE                 MAE                 AUC          
                       mean       std      mean       std      mean       std
Method                                                                       
Original           0.025964  0.000628  0.072601  0.001441  0.875089  0.002604
Oversampled        0.130376  0.008931  0.239528  0.018372  0.876320  0.002320
Oversampled-Corr   0.031746  0.002130  0.077420  0.002728  0.876320  0.002320
Undersampled       0.149042  0.018771  0.250910  0.024023  0.863073  0.011516
Undersampled-Corr  0.042101  0.012223  0.090242  0.013055  0.863073  0.011516
Weighted           0.130336  0.008816  0.239559  0.018018  0.876399  0.002216
Weighted-Corr      0.031705  0.002084  0.077375  0.002692  0.876399  0.002216

========== Outer Iteration 2/5 ==========



Outer Iteration 2 Progress: 100%|██████████| 500/500 [00:16<00:00, 31.08it/s]



=== Summary for Outer Iteration 2 ===

                        MSE                 MAE                 AUC          
                       mean       std      mean       std      mean       std
Method                                                                       
Original           0.014765  0.000531  0.038600  0.001996  0.978007  0.000790
Oversampled        0.052513  0.007526  0.095239  0.015183  0.977939  0.000853
Oversampled-Corr   0.015593  0.001048  0.036529  0.001788  0.977939  0.000853
Undersampled       0.075607  0.023312  0.112273  0.023515  0.968198  0.011015
Undersampled-Corr  0.036330  0.027381  0.057610  0.025137  0.968198  0.011015
Weighted           0.052653  0.007411  0.095566  0.014995  0.977959  0.000837
Weighted-Corr      0.015567  0.001025  0.036537  0.001749  0.977959  0.000837

========== Outer Iteration 3/5 ==========



...


=== Summary for Outer Iteration 3 ===

                        MSE                 MAE                 AUC          
                       mean       std      mean       std      mean       std
Method                                                                       
Original           0.016758  0.000646  0.054724  0.001179  0.919797  0.001347
Oversampled        0.098437  0.008473  0.187652  0.017826  0.919223  0.001797
Oversampled-Corr   0.020380  0.001537  0.055808  0.001657  0.919223  0.001797
Undersampled       0.113265  0.018783  0.196100  0.023944  0.910679  0.009079
Undersampled-Corr  0.030443  0.012245  0.067942  0.013270  0.910679  0.009079
Weighted           0.098504  0.008314  0.187890  0.017627  0.919288  0.001731
Weighted-Corr      0.020346  0.001550  0.055780  0.001638  0.919288  0.001731

========== Outer Iteration 4/5 ==========



Outer Iteration 4 Progress: 100%|██████████| 500/500 [00:13<00:00, 35.89it/s]



=== Summary for Outer Iteration 4 ===

                        MSE                 MAE                 AUC          
                       mean       std      mean       std      mean       std
Method                                                                       
Original           0.020312  0.000347  0.064017  0.001474  0.869011  0.002771
Oversampled        0.122285  0.010278  0.239946  0.022139  0.871347  0.001851
Oversampled-Corr   0.022374  0.001238  0.065149  0.001969  0.871347  0.001851
Undersampled       0.141569  0.021843  0.252606  0.028007  0.858416  0.011490
Undersampled-Corr  0.030601  0.009817  0.076759  0.012095  0.858416  0.011490
Weighted           0.122242  0.009998  0.240038  0.021621  0.871424  0.001801
Weighted-Corr      0.022333  0.001218  0.065098  0.001922  0.871424  0.001801

========== Outer Iteration 5/5 ==========



Outer Iteration 5 Progress: 100%|██████████| 500/500 [00:14<00:00, 33.79it/s]


=== Summary for Outer Iteration 5 ===

                        MSE                 MAE                 AUC          
                       mean       std      mean       std      mean       std
Method                                                                       
Original           0.028763  0.000824  0.078328  0.001287  0.858940  0.002419
Oversampled        0.149979  0.007635  0.265890  0.015986  0.859403  0.002066
Oversampled-Corr   0.037497  0.002549  0.086432  0.002685  0.859403  0.002066
Undersampled       0.169170  0.019451  0.276025  0.022785  0.845223  0.013068
Undersampled-Corr  0.050616  0.016297  0.101860  0.017072  0.845223  0.013068
Weighted           0.149984  0.007488  0.265898  0.015702  0.859496  0.001988
Weighted-Corr      0.037491  0.002538  0.086429  0.002666  0.859496  0.001988

=== Final summary over all outers and inners ===

                        MSE                 MAE                 AUC          
                       mean       std      mean    

In [10]:
#Simulation 2 - Multivariate Correlated Case

import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, mean_squared_error, mean_absolute_error
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from scipy.stats import multivariate_normal


p = 10                 
n_train = 1000
n_test = 5000
pi_plus = 0.05          
outer_iterations = 5
inner_iterations = 500

k1 = 2              
k0 = 3            

all_results = []

def generate_data_s2(n, mus_0, Sigmas_0, mus_1, Sigmas_1, pi_plus):
   
    n_minority = int(n * pi_plus)
    n_majority = n - n_minority

    #majority class (Y=0)
    comp_idx_0 = np.random.choice(k0, size=n_majority)
    X_majority = np.zeros((n_majority, p))
    for m in range(k0):
        idx = np.where(comp_idx_0 == m)[0]
        if len(idx) > 0:
            X_majority[idx, :] = np.random.multivariate_normal(
                mean=mus_0[m],
                cov=Sigmas_0[m],
                size=len(idx)
            )

    #minority class (Y=1)
    comp_idx_1 = np.random.choice(k1, size=n_minority)
    X_minority = np.zeros((n_minority, p))
    for m in range(k1):
        idx = np.where(comp_idx_1 == m)[0]
        if len(idx) > 0:
            X_minority[idx, :] = np.random.multivariate_normal(
                mean=mus_1[m],
                cov=Sigmas_1[m],
                size=len(idx)
            )

    X = np.vstack((X_majority, X_minority))
    y = np.hstack((np.zeros(n_majority), np.ones(n_minority)))
    return X, y



def compute_posterior_s2(X, mus_0, Sigmas_0, mus_1, Sigmas_1, pi_plus):
    n = X.shape[0]
    pi_minus = 1 - pi_plus

    #p(x | Y=1) = (1/k1) * sum_m phi(x; mu_1m, Sigma_1m)
    p_x_given_y1 = np.zeros(n)
    for m in range(k1):
        p_x_given_y1 += multivariate_normal.pdf(X, mean=mus_1[m], cov=Sigmas_1[m])
    p_x_given_y1 /= k1

    #p(x | Y=0) = (1/k0) * sum_m phi(x; mu_0m, Sigma_0m)
    p_x_given_y0 = np.zeros(n)
    for m in range(k0):
        p_x_given_y0 += multivariate_normal.pdf(X, mean=mus_0[m], cov=Sigmas_0[m])
    p_x_given_y0 /= k0

    # Bayes rule
    numerator = p_x_given_y1 * pi_plus
    denominator = numerator + p_x_given_y0 * pi_minus
    p_y1_given_x = numerator / denominator
    return p_y1_given_x



for outer in range(outer_iterations):
    print(f"\n========== Scenario 2: Outer Iteration {outer+1}/{outer_iterations} ==========\n")

    np.random.seed(outer)

    # Minority class - means from Unif(2,6)
    mus_1 = []
    Sigmas_1 = []
    for m in range(k1):
        mu_1m = np.random.uniform(2, 6, p)
        #random A, covariance = A A^T
        A = np.random.normal(0, 1, size=(p, p))
        Sigma_1m = A @ A.T
        mus_1.append(mu_1m)
        Sigmas_1.append(Sigma_1m)
    mus_1 = np.array(mus_1)

    # Majority class - means from Unif(0,4)
    mus_0 = []
    Sigmas_0 = []
    for m in range(k0):
        mu_0m = np.random.uniform(0, 4, p)
        A = np.random.normal(0, 1, size=(p, p))
        Sigma_0m = A @ A.T
        mus_0.append(mu_0m)
        Sigmas_0.append(Sigma_0m)
    mus_0 = np.array(mus_0)


    X_test, y_test = generate_data_s2(n_test, mus_0, Sigmas_0, mus_1, Sigmas_1, pi_plus)
    theoretical_probabilities = compute_posterior_s2(X_test, mus_0, Sigmas_0, mus_1, Sigmas_1, pi_plus)

    inner_results = []

   
    #Inner loop under fixed Test

    for inner in tqdm(range(inner_iterations), desc=f"Scenario 2 - Outer Iteration {outer+1} Progress"):
        X_train, y_train = generate_data_s2(n_train, mus_0, Sigmas_0, mus_1, Sigmas_1, pi_plus)

        #Train models: original, oversampled, undersampled, weighted
        model_orig = LogisticRegression(penalty=None, max_iter=1000).fit(X_train, y_train)

        X_over, y_over = RandomOverSampler(random_state=inner).fit_resample(X_train, y_train)
        model_over = LogisticRegression(penalty=None, max_iter=1000).fit(X_over, y_over)

        X_under, y_under = RandomUnderSampler(random_state=inner).fit_resample(X_train, y_train)
        model_under = LogisticRegression(penalty=None, max_iter=1000).fit(X_under, y_under)

        model_weighted = LogisticRegression(
            penalty=None, max_iter=1000, class_weight='balanced'
        ).fit(X_train, y_train)

     
        original_predicted_probabilities = model_orig.predict_proba(X_test)[:, 1]
        over_predicted_probabilities = model_over.predict_proba(X_test)[:, 1]
        under_predicted_probabilities = model_under.predict_proba(X_test)[:, 1]
        weighted_predicted_probabilities = model_weighted.predict_proba(X_test)[:, 1]

        #Probability Correction 
        proportion_0 = (y_train == 0).sum() / len(y_train)
        proportion_1 = (y_train == 1).sum() / len(y_train)

        # oversampled
        rate = proportion_0 / proportion_1
        adj_prob_over = over_predicted_probabilities / (
            rate - rate * over_predicted_probabilities + over_predicted_probabilities
        )

        # undersampled
        rate2 = proportion_1 / proportion_0
        adj_prob_under = (rate2 * under_predicted_probabilities) / (
            rate2 * under_predicted_probabilities - under_predicted_probabilities + 1
        )

        # weighted
        adj_prob_weighted = (weighted_predicted_probabilities / proportion_0) / (
            (1 / proportion_1)
            + (1 / proportion_0) * weighted_predicted_probabilities
            - (1 / proportion_1) * weighted_predicted_probabilities
        )

     
        mse_results = {
            "Method": [
                "Original",
                "Oversampled",
                "Oversampled-Corr",
                "Undersampled",
                "Undersampled-Corr",
                "Weighted",
                "Weighted-Corr",
            ],
            "MSE": [
                mean_squared_error(theoretical_probabilities, original_predicted_probabilities),
                mean_squared_error(theoretical_probabilities, over_predicted_probabilities),
                mean_squared_error(theoretical_probabilities, adj_prob_over),
                mean_squared_error(theoretical_probabilities, under_predicted_probabilities),
                mean_squared_error(theoretical_probabilities, adj_prob_under),
                mean_squared_error(theoretical_probabilities, weighted_predicted_probabilities),
                mean_squared_error(theoretical_probabilities, adj_prob_weighted),
            ],
            "MAE": [
                mean_absolute_error(theoretical_probabilities, original_predicted_probabilities),
                mean_absolute_error(theoretical_probabilities, over_predicted_probabilities),
                mean_absolute_error(theoretical_probabilities, adj_prob_over),
                mean_absolute_error(theoretical_probabilities, under_predicted_probabilities),
                mean_absolute_error(theoretical_probabilities, adj_prob_under),
                mean_absolute_error(theoretical_probabilities, weighted_predicted_probabilities),
                mean_absolute_error(theoretical_probabilities, adj_prob_weighted),
            ],
            "AUC": [
                roc_auc_score(y_test, original_predicted_probabilities),
                roc_auc_score(y_test, over_predicted_probabilities),
                roc_auc_score(y_test, adj_prob_over),
                roc_auc_score(y_test, under_predicted_probabilities),
                roc_auc_score(y_test, adj_prob_under),
                roc_auc_score(y_test, weighted_predicted_probabilities),
                roc_auc_score(y_test, adj_prob_weighted),
            ],
        }

        inner_results.append(pd.DataFrame(mse_results))


    df_results = pd.concat(inner_results)
    df_results["Outer"] = outer + 1
    summary = df_results.drop(columns=["Outer"]).groupby("Method").agg(["mean", "std"])

    print("\n=== Summary for Outer Iteration", outer+1, "===\n")
    print(summary)
    all_results.append(df_results)



df_all = pd.concat(all_results, ignore_index=True)[["Method", "MSE", "MAE", "AUC"]]

final_summary = df_all.groupby("Method").agg(["mean", "std"])
print("\n=== Final summary over all outers and inners ===\n")
print(final_summary)


========== Scenario 2: Outer Iteration 1/5 ==========



Scenario 2 - Outer Iteration 1 Progress: 100%|██████████| 500/500 [00:18<00:00, 26.81it/s]



=== Summary for Outer Iteration 1 ===

                        MSE                 MAE                 AUC          
                       mean       std      mean       std      mean       std
Method                                                                       
Original           0.027147  0.001344  0.052721  0.003714  0.945474  0.005400
Oversampled        0.083209  0.009503  0.139503  0.017279  0.950258  0.003831
Oversampled-Corr   0.034615  0.003367  0.059515  0.005834  0.950258  0.003831
Undersampled       0.110615  0.023289  0.154036  0.024332  0.934059  0.014572
Undersampled-Corr  0.061181  0.032204  0.083925  0.026235  0.934059  0.014572
Weighted           0.083532  0.009381  0.140145  0.017088  0.950365  0.003769
Weighted-Corr      0.034648  0.003310  0.059678  0.005754  0.950365  0.003769

========== Scenario 2: Outer Iteration 2/5 ==========



Scenario 2 - Outer Iteration 2 Progress: 100%|██████████| 500/500 [00:18<00:00, 27.74it/s]



=== Summary for Outer Iteration 2 ===

                        MSE                 MAE                 AUC          
                       mean       std      mean       std      mean       std
Method                                                                       
Original           0.031840  0.001579  0.064880  0.004402  0.860660  0.010913
Oversampled        0.133901  0.015091  0.263057  0.029573  0.871008  0.009794
Oversampled-Corr   0.039051  0.003436  0.074408  0.006142  0.871008  0.009794
Undersampled       0.163707  0.026933  0.282031  0.034488  0.854882  0.016893
Undersampled-Corr  0.051196  0.016227  0.091890  0.018609  0.854882  0.016893
Weighted           0.134248  0.014838  0.264107  0.028982  0.871416  0.009670
Weighted-Corr      0.039018  0.003363  0.074484  0.006039  0.871416  0.009670

========== Scenario 2: Outer Iteration 3/5 ==========



...


=== Summary for Outer Iteration 3 ===

                        MSE                 MAE                 AUC          
                       mean       std      mean       std      mean       std
Method                                                                       
Original           0.039755  0.000715  0.078613  0.002886  0.816451  0.012625
Oversampled        0.155802  0.013354  0.300967  0.028014  0.821733  0.010370
Oversampled-Corr   0.041697  0.001523  0.081125  0.003454  0.821733  0.010370
Undersampled       0.184266  0.025413  0.319525  0.032015  0.805395  0.018229
Undersampled-Corr  0.049951  0.009727  0.095823  0.013825  0.805395  0.018229
Weighted           0.155988  0.012960  0.301750  0.027448  0.822467  0.010100
Weighted-Corr      0.041628  0.001495  0.081105  0.003345  0.822467  0.010100

========== Scenario 2: Outer Iteration 4/5 ==========



Scenario 2 - Outer Iteration 4 Progress: 100%|██████████| 500/500 [00:18<00:00, 27.21it/s]



=== Summary for Outer Iteration 4 ===

                        MSE                 MAE                 AUC          
                       mean       std      mean       std      mean       std
Method                                                                       
Original           0.029248  0.001063  0.054743  0.003032  0.947247  0.004771
Oversampled        0.083631  0.009346  0.144758  0.019917  0.947433  0.004591
Oversampled-Corr   0.032829  0.002292  0.057019  0.003739  0.947433  0.004591
Undersampled       0.112405  0.025083  0.162102  0.026192  0.932079  0.015021
Undersampled-Corr  0.055856  0.030217  0.079983  0.025659  0.932079  0.015021
Weighted           0.083940  0.009176  0.145463  0.019589  0.947638  0.004499
Weighted-Corr      0.032772  0.002273  0.057084  0.003701  0.947638  0.004499

========== Scenario 2: Outer Iteration 5/5 ==========



Scenario 2 - Outer Iteration 5 Progress: 100%|██████████| 500/500 [00:19<00:00, 25.10it/s]



=== Summary for Outer Iteration 5 ===

                        MSE                 MAE                 AUC          
                       mean       std      mean       std      mean       std
Method                                                                       
Original           0.023070  0.001147  0.043399  0.002979  0.959867  0.003968
Oversampled        0.067901  0.009313  0.117525  0.019652  0.960882  0.003922
Oversampled-Corr   0.026378  0.002181  0.046211  0.004060  0.960882  0.003922
Undersampled       0.100229  0.026016  0.134308  0.027291  0.943963  0.016715
Undersampled-Corr  0.059020  0.034537  0.076371  0.028793  0.943963  0.016715
Weighted           0.068151  0.009233  0.118073  0.019558  0.960984  0.003917
Weighted-Corr      0.026360  0.002166  0.046287  0.004017  0.960984  0.003917

=== Final summary over all outers and inners ===

                        MSE                 MAE                 AUC          
                       mean       std      mean    

In [12]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from scipy.special import expit  # sigmoid


#Scenario 3 - high dimensional data case

def generate_data_s3(n, mu0_info, sigma0_info, mu1_info, sigma1_info, d, pi_plus):

    n_minority = int(n * pi_plus)
    n_majority = n - n_minority

    #majority (Y=0)
    X0_info = np.random.normal(mu0_info, sigma0_info, size=(n_majority, 5))
    X0_noise = np.random.normal(0, 1, size=(n_majority, d - 5))
    X0 = np.hstack((X0_info, X0_noise))

    #minority (Y=1)
    X1_info = np.random.normal(mu1_info, sigma1_info, size=(n_minority, 5))
    X1_noise = np.random.normal(0, 1, size=(n_minority, d - 5))
    X1 = np.hstack((X1_info, X1_noise))

    X = np.vstack((X0, X1))
    y = np.hstack((np.zeros(n_majority), np.ones(n_minority)))

    return X, y



def compute_true_posterior(X, mu0, sigma0, mu1, sigma1, pi_plus):
    #Only use first 5 informative features！！！！！！
    X_info = X[:, :5]

    #Gaussian density f0(x)
    f0 = np.prod(
        (1.0 / (np.sqrt(2 * np.pi) * sigma0)) *
        np.exp(-((X_info - mu0) ** 2) / (2 * sigma0 ** 2)),
        axis=1
    )

    #Gaussian density f1(x)
    f1 = np.prod(
        (1.0 / (np.sqrt(2 * np.pi) * sigma1)) *
        np.exp(-((X_info - mu1) ** 2) / (2 * sigma1 ** 2)),
        axis=1
    )

  
    numerator = pi_plus * f1
    denominator = numerator + (1 - pi_plus) * f0

    return numerator / denominator



#Refit
def refit_probability(scores, y_valid):
    model = LogisticRegression(
        penalty=None, max_iter=1000
    ).fit(scores.reshape(-1, 1), y_valid)
    return model

def get_pred_with_refit(model, refitter, X):
    score = model.decision_function(X).reshape(-1, 1)
    return refitter.predict_proba(score)[:, 1]


d_list = [50, 100, 200, 500]     # high-dimensional settings
p_info = 5           
pi_plus = 0.05
n_train = 1000
n_test = 5000
outer_iterations = 5
inner_iterations = 500

all_results = {}

for d in d_list:

    print(f"\n================= Scenario 3: Running d = {d} =================\n")

    dim_results = []

    for outer in range(outer_iterations):

        print(f"\n======= Outer Iteration {outer+1}/{outer_iterations} for d = {d} =======\n")
        np.random.seed(outer)

        #informative parameters
        mu0 = np.random.uniform(1, 5, p_info)
        sigma0 = np.random.uniform(2, 5, p_info)

        mu1 = np.random.uniform(4, 6, p_info)
        sigma1 = np.random.uniform(1, 2, p_info)

        #fixed test set
        X_test, y_test = generate_data_s3(n_test, mu0, sigma0, mu1, sigma1, d, pi_plus)
        p_true = compute_true_posterior(X_test, mu0, sigma0, mu1, sigma1, pi_plus)

        inner_list = []

        for inner in tqdm(range(inner_iterations), desc=f"d={d}, outer {outer+1}"):

            X_train, y_train = generate_data_s3(
                n_train, mu0, sigma0, mu1, sigma1, d, pi_plus
            )

            #split into train & valid
            X_tr, X_val, y_tr, y_val = train_test_split(
                X_train, y_train, test_size=0.2, stratify=y_train
            )

           
            # Original: L1 logistic on original data
            model_orig = LogisticRegression(
                penalty="l1", solver="liblinear",max_iter=1000
            ).fit(X_tr, y_tr)

            # Oversampled: L1 logistic on oversampled data
            X_over, y_over = RandomOverSampler(random_state=inner).fit_resample(X_tr, y_tr)
            model_over = LogisticRegression(
                penalty="l1", solver="liblinear", max_iter=1000
            ).fit(X_over, y_over)

            # Undersampled: L1 logistic on undersampled data
            X_under, y_under = RandomUnderSampler(random_state=inner).fit_resample(X_tr, y_tr)
            model_under = LogisticRegression(
                penalty="l1", solver="liblinear", max_iter=1000
            ).fit(X_under, y_under)

            # Weighted: L1 logistic with class_weight (on original data)
            model_weighted = LogisticRegression(
                penalty="l1", solver="liblinear",
                class_weight="balanced", max_iter=1000
            ).fit(X_tr, y_tr)

            p_orig_raw = model_orig.predict_proba(X_test)[:, 1]
            p_over_raw = model_over.predict_proba(X_test)[:, 1]
            p_under_raw = model_under.predict_proba(X_test)[:, 1]
            p_weighted_raw = model_weighted.predict_proba(X_test)[:, 1]

            # Step 2: refit on validation 
            scores_orig = model_orig.decision_function(X_val)
            refit_orig = refit_probability(scores_orig, y_val)

            scores_over = model_over.decision_function(X_val)
            refit_over = refit_probability(scores_over, y_val)

            scores_under = model_under.decision_function(X_val)
            refit_under = refit_probability(scores_under, y_val)

            scores_weighted = model_weighted.decision_function(X_val)
            refit_weighted = refit_probability(scores_weighted, y_val)

            p_orig_refit = get_pred_with_refit(model_orig, refit_orig, X_test)
            p_over_refit = get_pred_with_refit(model_over, refit_over, X_test)
            p_under_refit = get_pred_with_refit(model_under, refit_under, X_test)
            p_weighted_refit = get_pred_with_refit(model_weighted, refit_weighted, X_test)


            # Step 3: correction
            prop1 = y_train.mean()
            prop0 = 1 - prop1

            # oversample correction on refit prob
            rate = prop0 / prop1
            p_over_refit_corr = p_over_refit / (rate - rate * p_over_refit + p_over_refit)

            # undersample correction on refit prob
            rate2 = prop1 / prop0
            p_under_refit_corr = (rate2 * p_under_refit) / (
                rate2 * p_under_refit - p_under_refit + 1
            )

            # weighted correction on refit prob
            p_weighted_refit_corr = (p_weighted_refit / prop0) / (
                (1 / prop1)
                + (1 / prop0) * p_weighted_refit
                - (1 / prop1) * p_weighted_refit
            )

            mse_results = pd.DataFrame({
                "Method": [
                    "Original", "Original+Refit",
                    "Oversampled", "Oversampled+Refit", "Oversampled+Refit+Corr",
                    "Undersampled", "Undersampled+Refit", "Undersampled+Refit+Corr",
                    "Weighted", "Weighted+Refit", "Weighted+Refit+Corr"
                ],
                "MSE": [
                    mean_squared_error(p_true, p_orig_raw),
                    mean_squared_error(p_true, p_orig_refit),

                    mean_squared_error(p_true, p_over_raw),
                    mean_squared_error(p_true, p_over_refit),
                    mean_squared_error(p_true, p_over_refit_corr),

                    mean_squared_error(p_true, p_under_raw),
                    mean_squared_error(p_true, p_under_refit),
                    mean_squared_error(p_true, p_under_refit_corr),

                    mean_squared_error(p_true, p_weighted_raw),
                    mean_squared_error(p_true, p_weighted_refit),
                    mean_squared_error(p_true, p_weighted_refit_corr),
                ],
                "MAE": [
                    mean_absolute_error(p_true, p_orig_raw),
                    mean_absolute_error(p_true, p_orig_refit),

                    mean_absolute_error(p_true, p_over_raw),
                    mean_absolute_error(p_true, p_over_refit),
                    mean_absolute_error(p_true, p_over_refit_corr),

                    mean_absolute_error(p_true, p_under_raw),
                    mean_absolute_error(p_true, p_under_refit),
                    mean_absolute_error(p_true, p_under_refit_corr),

                    mean_absolute_error(p_true, p_weighted_raw),
                    mean_absolute_error(p_true, p_weighted_refit),
                    mean_absolute_error(p_true, p_weighted_refit_corr),
                ],
                "AUC": [
                    roc_auc_score(y_test, p_orig_raw),
                    roc_auc_score(y_test, p_orig_refit),

                    roc_auc_score(y_test, p_over_raw),
                    roc_auc_score(y_test, p_over_refit),
                    roc_auc_score(y_test, p_over_refit_corr),

                    roc_auc_score(y_test, p_under_raw),
                    roc_auc_score(y_test, p_under_refit),
                    roc_auc_score(y_test, p_under_refit_corr),

                    roc_auc_score(y_test, p_weighted_raw),
                    roc_auc_score(y_test, p_weighted_refit),
                    roc_auc_score(y_test, p_weighted_refit_corr),
                ],
            })

            inner_list.append(mse_results)

        df_dim_outer = pd.concat(inner_list)
        df_dim_outer["Outer"] = outer + 1

        print("\n--- Summary for this outer ---\n")
        print(df_dim_outer.groupby("Method").agg(["mean", "std"]))

        dim_results.append(df_dim_outer)

    # collect results for this dimension d
    all_results[d] = pd.concat(dim_results)



print("\n==================== FINAL SUMMARY FOR ALL DIMENSIONS ====================\n")

for d in d_list:
    print(f"\n=== d = {d} ===\n")
    print(all_results[d].groupby("Method").agg(["mean", "std"]))


================= Scenario 3: Running d = 50 =================


======= Outer Iteration 1/5 for d = 50 =======



d=50, outer 1: 100%|██████████| 500/500 [00:57<00:00,  8.72it/s]



--- Summary for this outer ---

                              MSE                 MAE                 AUC  \
                             mean       std      mean       std      mean   
Method                                                                      
Original                 0.030717  0.001881  0.081009  0.002465  0.739187   
Original+Refit           0.025922  0.001140  0.080287  0.002522  0.739187   
Oversampled              0.135578  0.009901  0.219024  0.017911  0.754913   
Oversampled+Refit        0.025930  0.001319  0.079828  0.002643  0.754913   
Oversampled+Refit+Corr   0.028078  0.000064  0.049637  0.000186  0.754913   
Undersampled             0.280721  0.039742  0.395215  0.040968  0.700045   
Undersampled+Refit       0.026102  0.000961  0.081888  0.002437  0.697111   
Undersampled+Refit+Corr  0.028153  0.000060  0.049781  0.000160  0.697111   
Weighted                 0.136269  0.009793  0.224214  0.017798  0.757749   
Weighted+Refit           0.025935  0.001329

d=50, outer 2: 100%|██████████| 500/500 [01:43<00:00,  4.83it/s]



--- Summary for this outer ---

                              MSE                 MAE                 AUC  \
                             mean       std      mean       std      mean   
Method                                                                      
Original                 0.022030  0.001952  0.051077  0.002980  0.938452   
Original+Refit           0.020604  0.002055  0.054677  0.004741  0.938452   
Oversampled              0.050674  0.006904  0.080294  0.010373  0.931239   
Oversampled+Refit        0.021492  0.002152  0.056872  0.004875  0.931239   
Oversampled+Refit+Corr   0.029587  0.001428  0.046898  0.000873  0.931239   
Undersampled             0.146284  0.030924  0.240439  0.036507  0.910050   
Undersampled+Refit       0.024705  0.002910  0.064064  0.007212  0.910050   
Undersampled+Refit+Corr  0.030880  0.001170  0.048165  0.000875  0.910050   
Weighted                 0.050575  0.006962  0.084810  0.010757  0.936184   
Weighted+Refit           0.021012  0.002176

...


--- Summary for this outer ---

                              MSE                 MAE                 AUC  \
                             mean       std      mean       std      mean   
Method                                                                      
Original                 0.029002  0.002179  0.057240  0.003248  0.902938   
Original+Refit           0.024055  0.001963  0.064258  0.005115  0.902938   
Oversampled              0.046459  0.004690  0.075163  0.005761  0.887254   
Oversampled+Refit        0.024957  0.001898  0.067026  0.005145  0.887254   
Oversampled+Refit+Corr   0.030726  0.000858  0.048122  0.000499  0.887254   
Undersampled             0.217039  0.038813  0.330364  0.040962  0.854717   
Undersampled+Refit       0.027434  0.002206  0.073219  0.006339  0.854717   
Undersampled+Refit+Corr  0.031564  0.000609  0.048956  0.000506  0.854717   
Weighted                 0.044871  0.004530  0.076994  0.005854  0.895052   
Weighted+Refit           0.024518  0.001934

d=200, outer 3: 100%|██████████| 500/500 [02:20<00:00,  3.55it/s]



--- Summary for this outer ---

                              MSE                 MAE                 AUC  \
                             mean       std      mean       std      mean   
Method                                                                      
Original                 0.027756  0.002415  0.058675  0.003694  0.871984   
Original+Refit           0.021530  0.001743  0.063416  0.004705  0.871984   
Oversampled              0.047344  0.005535  0.079200  0.006732  0.855371   
Oversampled+Refit        0.022420  0.001686  0.065910  0.004678  0.855371   
Oversampled+Refit+Corr   0.028095  0.000690  0.048675  0.000429  0.855371   
Undersampled             0.231395  0.045375  0.349431  0.048389  0.824998   
Undersampled+Refit       0.024341  0.002260  0.071431  0.005980  0.824998   
Undersampled+Refit+Corr  0.028544  0.000665  0.049365  0.000541  0.824998   
Weighted                 0.045833  0.005333  0.080881  0.006834  0.863124   
Weighted+Refit           0.022019  0.001701

d=200, outer 4:  65%|██████▌   | 326/500 [02:21<01:15,  2.31it/s]


KeyboardInterrupt: 

In [34]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    brier_score_loss,
    log_loss,
)
from sklearn.calibration import calibration_curve
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.metrics import geometric_mean_score


def calculate_ece(y_true, y_prob, n_bins=10):
    prob_true, prob_pred = calibration_curve(y_true, y_prob, n_bins=n_bins)
    return np.mean(np.abs(prob_true - prob_pred))


def calculate_gmean(y_true, y_prob, threshold):
    y_pred = (y_prob >= threshold).astype(int)
    return geometric_mean_score(y_true, y_pred)


def run_balancing_experiment(
    X,
    y,
    n_iterations=100,
    test_size=0.3,
    subtrain_ratio=0.8,
    random_state=1005
):

    X = np.asarray(X)
    y = np.asarray(y)

    # fixed test set
    X_train_full, X_test, y_train_full, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    methods = [
        "Original",
        "Oversampled", "Oversampled-Adj",
        "Undersampled", "Undersampled-Adj",
        "Weighted", "Weighted-Adj",
    ]

    metrics = ["AUC", "Brier", "LogLoss", "ECE", "Gmean"]

   
    results = {metric: {m: [] for m in methods} for metric in metrics}

    
    for it in range(n_iterations):

        # random subset: 80% of training as true training
        X_tr, _, y_tr, _ = train_test_split(
            X_train_full, y_train_full,
            test_size=1 - subtrain_ratio,
            random_state=it,
            stratify=y_train_full
        )

      
        X_over, y_over = RandomOverSampler(random_state=it).fit_resample(X_tr, y_tr)
        X_under, y_under = RandomUnderSampler(random_state=it).fit_resample(X_tr, y_tr)

        
        clf_orig = LogisticRegression(max_iter=1000).fit(X_tr, y_tr)
        clf_over = LogisticRegression(max_iter=1000).fit(X_over, y_over)
        clf_under = LogisticRegression(max_iter=1000).fit(X_under, y_under)
        clf_weight = LogisticRegression(max_iter=1000, class_weight="balanced").fit(X_tr, y_tr)

        
        p_orig = clf_orig.predict_proba(X_test)[:, 1]
        p_over = clf_over.predict_proba(X_test)[:, 1]
        p_under = clf_under.predict_proba(X_test)[:, 1]
        p_weight = clf_weight.predict_proba(X_test)[:, 1]

       
        prop1 = y_tr.mean()
        prop0 = 1 - prop1

       
        rate = prop0 / prop1
        p_over_adj = np.clip(p_over / (rate - rate * p_over + p_over), 0,1)

        rate2 = prop1 / prop0
        p_under_adj = np.clip((rate2 * p_under) / (rate2 * p_under - p_under + 1),0,1)

        p_weight_adj = np.clip((p_weight / prop0) / (
            (1 / prop1) + (1 / prop0) * p_weight - (1 / prop1) * p_weight
        ),0,1)

        
        def add_all_metrics(name, y_prob, adjusted=False):
            results["AUC"][name].append(roc_auc_score(y_test, y_prob))
            results["Brier"][name].append(brier_score_loss(y_test, y_prob))
            results["LogLoss"][name].append(log_loss(y_test, y_prob))
            results["ECE"][name].append(calculate_ece(y_test, y_prob))

            thr = prop1 if adjusted else 0.5
            results["Gmean"][name].append(calculate_gmean(y_test, y_prob, thr))

        
        add_all_metrics("Original", p_orig)


        add_all_metrics("Oversampled", p_over)
        add_all_metrics("Oversampled-Adj", p_over_adj, adjusted=True)


        add_all_metrics("Undersampled", p_under)
        add_all_metrics("Undersampled-Adj", p_under_adj, adjusted=True)


        add_all_metrics("Weighted", p_weight)
        add_all_metrics("Weighted-Adj", p_weight_adj, adjusted=True)


    mean_table = pd.DataFrame({
        metric: {m: np.mean(vals) for m, vals in results[metric].items()}
        for metric in metrics
    })

    std_table = pd.DataFrame({
        metric: {m: np.std(vals, ddof=1) for m, vals in results[metric].items()}
        for metric in metrics
    })

    return mean_table, std_table









In [35]:
from ucimlrepo import fetch_ucirepo 

#Application 1

letter_recognition = fetch_ucirepo(id=59) 
X = letter_recognition.data.features 
y = letter_recognition.data.targets['lettr'] 
Y = y.apply(lambda letter: 1 if letter == 'A' else 0)


mean_tbl, std_tbl = run_balancing_experiment(X, Y)

print(mean_tbl)
print(std_tbl)

                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.990420  0.009834  0.037728  0.095350  0.910605
Oversampled       0.989506  0.037585  0.143134  0.371003  0.962198
Oversampled-Adj   0.989506  0.013813  0.050093  0.084215  0.962198
Undersampled      0.988610  0.042273  0.159092  0.381254  0.957356
Undersampled-Adj  0.988610  0.014570  0.053905  0.083982  0.957356
Weighted          0.989533  0.037537  0.142950  0.370673  0.962465
Weighted-Adj      0.989533  0.013807  0.050009  0.084561  0.962465
                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.000299  0.000206  0.000632  0.017665  0.003192
Oversampled       0.000280  0.001564  0.004594  0.003619  0.001816
Oversampled-Adj   0.000280  0.000285  0.000954  0.011744  0.001816
Undersampled      0.001277  0.004669  0.018470  0.012537  0.004373
Undersampled-Adj  0.001277  0.001568  0.006826  0.020679  0.004373
Weighted          0.000257  0.001562  0.004593  0.003664  0.00

In [36]:
#Application 2

letter_recognition = fetch_ucirepo(id=59) 
  
X = letter_recognition.data.features 
y = letter_recognition.data.targets['lettr'] 
vowels = {'A', 'E', 'I', 'O', 'U'}
Y = y.apply(lambda letter: 1 if letter in vowels else 0)


mean_tbl, std_tbl = run_balancing_experiment(X, Y)

print(mean_tbl)
print(std_tbl)

                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.749525  0.140127  0.436853  0.203429  0.341932
Oversampled       0.758058  0.205439  0.610433  0.289543  0.698512
Oversampled-Adj   0.758058  0.142953  0.441732  0.247152  0.698512
Undersampled      0.757381  0.206116  0.612583  0.288454  0.698131
Undersampled-Adj  0.757381  0.143376  0.442706  0.248490  0.698131
Weighted          0.758339  0.205382  0.610469  0.289657  0.698858
Weighted-Adj      0.758339  0.142948  0.441734  0.247396  0.698858
                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.000972  0.000128  0.000281  0.007401  0.008023
Oversampled       0.000833  0.000780  0.001484  0.002064  0.002174
Oversampled-Adj   0.000833  0.000301  0.000769  0.006111  0.002174
Undersampled      0.001844  0.001252  0.003626  0.002780  0.003505
Undersampled-Adj  0.001844  0.000750  0.001791  0.015125  0.003505
Weighted          0.000609  0.000574  0.001215  0.001707  0.00

In [37]:
#Application 3
page_blocks_classification = fetch_ucirepo(id=78) 
  
X = page_blocks_classification.data.features 
y = page_blocks_classification.data['targets'] 

if isinstance(y, pd.DataFrame):
    y = y.iloc[:, 0]


Y = y.apply(lambda value: 0 if value == 1 else 1)


mean_tbl, std_tbl = run_balancing_experiment(X, Y)

print(mean_tbl)
print(std_tbl)

/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.944160  0.038775  0.158084  0.130701  0.774470
Oversampled       0.968774  0.064123  0.283010  0.259992  0.917846
Oversampled-Adj   0.968774  0.036419  0.165214  0.103731  0.917846
Undersampled      0.967666  0.069393  0.343981  0.247772  0.910353
Undersampled-Adj  0.967666  0.042046  0.214874  0.143205  0.910353
Weighted          0.968683  0.064705  0.287513  0.256098  0.916277
Weighted-Adj      0.968683  0.036894  0.167149  0.110640  0.916277
                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.009303  0.000906  0.013702  0.013697  0.011719
Oversampled       0.004826  0.002535  0.010695  0.010138  0.008310
Oversampled-Adj   0.004826  0.001006  0.011338  0.019760  0.008310
Undersampled      0.007296  0.005731  0.044105  0.025644  0.016933
Undersampled-Adj  0.007296  0.003670  0.043078  0.034227  0.016933
Weighted          0.004126  0.002323  0.010173  0.008032  0.00

/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
#Application 4
covertype = fetch_ucirepo(id=31)


X = covertype.data.features  
y = covertype.data.original['Cover_Type'] 

Y = y.apply(lambda value: 1 if value == 4 else 0)


mean_tbl, std_tbl = run_balancing_experiment(X, Y)

print(mean_tbl)
print(std_tbl)

/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

KeyboardInterrupt: 

In [38]:
from sklearn.preprocessing import LabelEncoder

#Application 5

car_evaluation = fetch_ucirepo(id=19) 
  
X = car_evaluation.data.features 
y = car_evaluation.data.targets['class'] 

label_encoder = LabelEncoder()
X = X.apply(label_encoder.fit_transform)

Y = y.apply(lambda res: 1 if res == 'vgood' else 0)


mean_tbl, std_tbl = run_balancing_experiment(X, Y)

print(mean_tbl)
print(std_tbl)

                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.941361  0.031298  0.101984  0.156562  0.044571
Oversampled       0.942785  0.094805  0.304283  0.390831  0.909288
Oversampled-Adj   0.942785  0.034513  0.105550  0.244346  0.909288
Undersampled      0.932198  0.123848  0.378764  0.405308  0.890862
Undersampled-Adj  0.932198  0.033088  0.109519  0.204904  0.890862
Weighted          0.942896  0.095698  0.303920  0.390697  0.909791
Weighted-Adj      0.942896  0.033659  0.103821  0.246199  0.909791
                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.002297  0.000320  0.000947  0.032305  0.089591
Oversampled       0.001316  0.003150  0.010779  0.007482  0.007763
Oversampled-Adj   0.001316  0.000809  0.002017  0.036042  0.007763
Undersampled      0.008080  0.014643  0.044541  0.013363  0.017233
Undersampled-Adj  0.008080  0.001469  0.004689  0.073413  0.017233
Weighted          0.001357  0.002915  0.009664  0.006206  0.00

In [39]:
#Application 6

cirrhosis_patient_survival_prediction = fetch_ucirepo(id=878) 
  
# data (as pandas dataframes) 
X = cirrhosis_patient_survival_prediction.data.features 
label_encoder = LabelEncoder()
X = X.apply(label_encoder.fit_transform)
y = cirrhosis_patient_survival_prediction.data.targets['Status']

Y = y.apply(lambda res: 1 if res == 'CL' else 0)


mean_tbl, std_tbl = run_balancing_experiment(X, Y)

print(mean_tbl)
print(std_tbl)

/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.725413  0.071692  0.265910  0.316415  0.343743
Oversampled       0.709915  0.192236  0.629125  0.410954  0.618259
Oversampled-Adj   0.709915  0.078201  0.306087  0.356155  0.618259
Undersampled      0.672865  0.362580  4.851141  0.464216  0.624034
Undersampled-Adj  0.672865  0.284324  4.117990  0.451161  0.624034
Weighted          0.721123  0.192412  0.623280  0.408180  0.640272
Weighted-Adj      0.721123  0.076381  0.293012  0.341511  0.640272
                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.029334  0.005261  0.020022  0.044122  0.068440
Oversampled       0.042050  0.017138  0.053614  0.013311  0.086123
Oversampled-Adj   0.042050  0.009280  0.040404  0.044503  0.086123
Undersampled      0.061550  0.077609  3.676266  0.053708  0.070679
Undersampled-Adj  0.061550  0.107298  3.668761  0.051892  0.070679
Weighted          0.040771  0.016488  0.050637  0.013758  0.07

/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

In [41]:
#Application 7

aids_clinical_trials_group_study_175 = fetch_ucirepo(id=890) 
 
X = aids_clinical_trials_group_study_175.data.features 
Y = aids_clinical_trials_group_study_175.data.targets['cid']


mean_tbl, std_tbl = run_balancing_experiment(X, Y)

print(mean_tbl)
print(std_tbl)

/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modul

                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.868324  0.105153  0.354235  0.071879  0.707339
Oversampled       0.875020  0.119917  0.393611  0.132693  0.806968
Oversampled-Adj   0.875020  0.107107  0.354765  0.071981  0.806968
Undersampled      0.874344  0.122472  0.400818  0.137638  0.806456
Undersampled-Adj  0.874344  0.107919  0.356550  0.071274  0.806456
Weighted          0.876395  0.119546  0.392825  0.130586  0.806170
Weighted-Adj      0.876395  0.106777  0.353084  0.074470  0.806170
                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.002888  0.000851  0.002588  0.009614  0.011080
Oversampled       0.003592  0.002274  0.005577  0.008813  0.007876
Oversampled-Adj   0.003592  0.001108  0.003121  0.013653  0.007876
Undersampled      0.005126  0.002622  0.006949  0.011121  0.008112
Undersampled-Adj  0.005126  0.001615  0.004956  0.014606  0.008112
Weighted          0.002861  0.001665  0.004060  0.007055  0.00

/Users/hezhilin/anaconda3/envs/CMU16720/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [40]:
#Application 8

data_path = '/Users/hezhilin/Desktop/Code/creditcard.csv'
data = pd.read_csv(data_path)

X = data.iloc[:, :-1]
Y = data.iloc[:, -1]

mean_tbl, std_tbl = run_balancing_experiment(X, Y)

print(mean_tbl)
print(std_tbl)

                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.940828  0.000788  0.006466  0.152572  0.805121
Oversampled       0.968602  0.040517  0.173593  0.478462  0.932168
Oversampled-Adj   0.968602  0.000959  0.008520  0.291109  0.932168
Undersampled      0.966952  0.046738  0.199780  0.480666  0.929064
Undersampled-Adj  0.966952  0.001405  0.011581  0.367688  0.929064
Weighted          0.968440  0.040461  0.173590  0.478140  0.932217
Weighted-Adj      0.968440  0.000946  0.008575  0.291284  0.932217
                       AUC     Brier   LogLoss       ECE     Gmean
Original          0.011028  0.000119  0.001505  0.041243  0.011919
Oversampled       0.007132  0.007956  0.036199  0.002249  0.007617
Oversampled-Adj   0.007132  0.000246  0.002170  0.089044  0.007617
Undersampled      0.007671  0.008083  0.035372  0.006091  0.007849
Undersampled-Adj  0.007671  0.000662  0.004202  0.069704  0.007849
Weighted          0.006685  0.008548  0.037999  0.002537  0.00